In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import calc_footprint_FFP_climatology as ffp
import cv2
from affine import Affine
import rasterio
import pyproj as proj
import gc
import os
from datetime import datetime

In [18]:
def write_img(f_2d, x_2d, y_2d,name):
    affine_transform = find_transform(x_2d, y_2d)
    # Create the GeoTiff file
    out_f = os.path.join(f'tiffs/{name}.tif')
    with rasterio.open(out_f, 'w', driver='GTiff', dtype=rasterio.float64,
                       height=f_2d.shape[0], width=f_2d.shape[1],
                       count=count, transform=affine_transform, crs=out_proj.srs,
                       nodata=0.00000000e+000) as new_dat:
        new_dat.write(f_2d, 1)
    new_dat.close()

def process_row(row):
    temp_ffp =  ffp.FFP_climatology(
    zm=row.zm - row.d,
    z0=row.z0,
    umean=row.u_mean,
    h=2000,
    ol=row.L,
    sigmav=row.sigma_v,
    ustar=row.u_star,
    wind_dir=row.wind_dir,
    fig=True,
    domain=[-origin_d,origin_d, -origin_d,origin_d],
    dx=dx,
    dy=dx,
    verbosity=1
    )
    f_2d = np.array(temp_ffp['fclim_2d'])
    x_2d = np.array(temp_ffp['x_2d']) + station_x
    y_2d = np.array(temp_ffp['y_2d']) + station_y
    mask = np.zeros_like(f_2d)
    if temp_ffp['fr'][-2] !=None:
        mask[f_2d>temp_ffp['fr'][-2]] = 1
        # plt.imshow(mask)
        # f_2d = f_2d*dx**2
        # At the beginning of your function, print the index of the row:
        # print(f"Processing row with index {row.name}")

        # Rest of your processing code...        
        return mask, x_2d, y_2d

In [2]:
def find_transform(xs,ys):
    '''
    Returns the affine transform for 2d arrays xs and ys
    
    Args:
        xs (float) : 2D numpy array of x-coordinates
        ys (float) : 2D numpy array of y-coordinates
        
    Returns:
        aff_transform : affine.Affine object  
    '''
    
    shape = xs.shape

    #Choose points to calculate affine transform
    y_points = [0, 0, shape[0] - 1]
    x_points = [0, shape[0] - 1, shape[1] - 1]
    in_xy = np.float32([[i, j] for i, j in zip(x_points, y_points)])
    out_xy = np.float32([[xs[i, j], ys[i, j]] for i, j in zip(y_points, x_points)])
    

    #Calculate affine transform
    aff_transform = Affine(*cv2.getAffineTransform(in_xy,out_xy).flatten())

    return aff_transform

In [3]:
def get_var(file, var_name):
    df = pd.read_csv(file, index_col=0, parse_dates=True)
    var = df[var_name]
    del df
    gc.collect()
    return var

In [6]:
v_rot = get_var("flux_more_original.csv", ["v_var", "L"])

In [7]:
data = pd.read_excel('processed_data.xlsx', sheet_name='data',index_col=0,parse_dates=True)
data.columns
data = data.loc[:,['d','z0','u','ustar','WD(1)','hc']]
data = data.join(v_rot)

In [9]:
data = data.rename(columns={'WD(1)':'wind_dir','hc':'zm','u':'u_mean','ustar':'u_star'})
data['sigma_v'] = data['v_var']**0.5

In [23]:
dx=3
origin_d = 750.
station = 'crk'
count = 1

In [8]:
# daily = data.resample('D').mean()
# monthly = daily.resample('Y').mean()
# monthly['sigma_v'] = daily['v_unrot'].resample('Y').std()
# monthly.rename(columns={'WD(1)':'wind_dir','hc':'h_canopy','u':'u_mean','ustar':'u_star'},inplace=True)
# monthly

In [11]:
latitude,longitude = 38.20139, 127.25056
station_coord = (longitude, latitude)

EPSG=32700-np.round((45+latitude)/90.0)*100+np.round((183+longitude)/6.0)
EPSG = int(EPSG)
in_proj = proj.Proj(init='EPSG:4326')
out_proj = proj.Proj(init='EPSG:{}'.format(EPSG))
(station_x,station_y) = proj.transform(in_proj,out_proj,*station_coord)

/bess23/huaize/github/flux-data-footprint/.venv/lib/python3.8/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/bess23/huaize/github/flux-data-footprint/.venv/lib/python3.8/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/tmp/ipykernel_1762096/4165491826.py:8: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to

In [12]:
data.loc[data.index.year<2021,'zm'] = 10
data.loc[data.index.year>=2021,'zm'] = 5

## Daily

In [13]:
data =data[data.index.year>2015].dropna()

In [28]:
data[data.index.date==datetime(2016,1,10
).date()]

,d,z0,u_mean,u_star,wind_dir,zm,v_var,L,sigma_v
2016-01-10 00:00:00,0.0,0.00001,0.820624,0.163390,96.302229,10.0,0.230919,139.709082,0.480540
2016-01-10 00:30:00,0.0,0.00001,0.874853,0.173800,213.014714,10.0,0.475057,34.768690,0.689244
2016-01-10 01:00:00,0.0,0.00001,1.279415,0.184210,222.022503,10.0,1.169319,75.144027,1.081351
2016-01-10 01:30:00,0.0,0.00001,0.907525,0.157785,109.885438,10.0,0.168967,0.076900,0.411056
2016-01-10 02:00:00,0.0,0.00001,0.436276,0.131360,42.993781,10.0,0.659269,19.542596,0.811954
2016-01-10 02:30:00,0.0,0.00001,1.188213,0.131905,207.560306,10.0,0.648685,27.967755,0.805410
2016-01-10 03:00:00,0.0,0.00001,0.849766,0.132450,237.836280,10.0,0.978303,124.925977,0.989092
2016-01-10 03:30:00,0.0,0.00001,0.474134,0.296860,77.135695,10.0,0.633398,-94.987300,0.795863
2016-01-10 04:00:00,0.0,0.00001,1.819192,0.195730,12.277262,10.0,0.427561,37.175191,0.653882
2016-01-10 04:30:00,0.0,0.00001,1.188232,0.098700,43.341059,10.0,0.265689,17.859210,0.515450


In [25]:
results

NameError: name 'results' is not defined

In [30]:
data_grouped = data.groupby(data.index.date)
for date, group in data_grouped:
    try:
        results = group.apply(process_row, axis=1)
    except IndexError as e:
        continue
    f_2d_avg = np.mean([item[0] for item in results if item is not None], axis=0)
    x_2d_avg = np.mean([item[1] for item in results if item is not None], axis=0)
    y_2d_avg = np.mean([item[2] for item in results if item is not None], axis=0)
    write_img(f_2d_avg, x_2d_avg, y_2d_avg,date)
    del results, f_2d_avg, x_2d_avg, y_2d_avg,group
    gc.collect()

## Monthly

In [ ]:
data = ori_data
# 改为按月分组
data_grouped = data.groupby(data.index.to_period('M'))

for period, group in data_grouped:
    results = group.apply(process_row, axis=1)
    f_2d_avg = np.mean([item[0] for item in results if item is not None], axis=0)
    x_2d_avg = np.mean([item[1] for item in results if item is not None], axis=0)
    y_2d_avg = np.mean([item[2] for item in results if item is not None], axis=0)
    # 由于 period 是 Period 对象，使用 period.start_time 获取月份的起始日期
    write_img(f_2d_avg, x_2d_avg, y_2d_avg, period.start_time)